In [1]:
# Reading the wiki page and scrapping the table information 
import numpy as np
import pandas as pd
from pandas.io.html import read_html
link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(link,header=0)[0] # 0 represents the 1st table in the link and 1 represent the 2nd table

tables

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [2]:
# Converting to pandas dataframe 
PC_can=pd.DataFrame(tables)
PC_can.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
# Drop rows where borough=Not assigned
PC_cano=PC_can[PC_can.Borough!='Not assigned']
PC_cano

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
...,...,...,...
282,M8Z,Etobicoke,Kingsway Park South West
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West


In [4]:
df = (PC_cano['Neighbourhood'].str.split(', ') #More than one neighborhood can exist in one postal code area.These two rows will be combined into one row with the neighborhoods separated with a comma
                    .groupby(PC_cano['Postcode'])
                    .agg({ 'Neighbourhood':lambda x: ', '.join(set(y for z in x for y in z))})
                    .reset_index())
df


/Users/upasanadhar/anaconda/miniconda3/envs/mlcourse/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  This is separate from the ipykernel package so we can avoid doing imports until


,Postcode,Neighbourhood
0,M1B,"Rouge, Malvern"
1,M1C,"Port Union, Highland Creek, Rouge Hill"
2,M1E,"West Hill, Morningside, Guildwood"
3,M1G,Woburn
4,M1H,Cedarbrae
...,...,...
98,M9N,Weston
99,M9P,Westmount
100,M9R,"Kingsview Village, Richview Gardens, St. Phill..."
101,M9V,"Silverstone, South Steeles, Beaumond Heights, ..."


In [5]:
dfr=df.join(PC_cano.set_index('Postcode')[['Borough']], on='Postcode') # Adding Borough from PC_cano dataset to the df dataset with similar Postcodes
dfr # generates new dataset but with duplicate values of Postcode since neighbourhod was not split and joined in 1st dataframe
dfr = dfr.drop_duplicates(subset='Postcode', keep='first')#Dropping duplicate postcodes
dfr

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Port Union, Highland Creek, Rouge Hill",Scarborough
2,M1E,"West Hill, Morningside, Guildwood",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
...,...,...,...
98,M9N,Weston,York
99,M9P,Westmount,Etobicoke
100,M9R,"Kingsview Village, Richview Gardens, St. Phill...",Etobicoke
101,M9V,"Silverstone, South Steeles, Beaumond Heights, ...",Etobicoke


In [6]:
dfr.loc[dfr['Neighbourhood'] == 'Not assigned']

,Postcode,Neighbourhood,Borough
85,M7A,Not assigned,Queen's Park


In [7]:
dfr_copy=dfr.copy()
dfr_copy.loc[dfr['Neighbourhood']=='Not assigned', 'Neighbourhood'] = dfr['Borough']
dfr_copy

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Port Union, Highland Creek, Rouge Hill",Scarborough
2,M1E,"West Hill, Morningside, Guildwood",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
...,...,...,...
98,M9N,Weston,York
99,M9P,Westmount,Etobicoke
100,M9R,"Kingsview Village, Richview Gardens, St. Phill...",Etobicoke
101,M9V,"Silverstone, South Steeles, Beaumond Heights, ...",Etobicoke


In [8]:
# Chnage name of column Postcode to Postal code
df1=dfr_copy.rename(columns={"Postcode": "Postal Code"})
df1

,Postal Code,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Port Union, Highland Creek, Rouge Hill",Scarborough
2,M1E,"West Hill, Morningside, Guildwood",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
...,...,...,...
98,M9N,Weston,York
99,M9P,Westmount,Etobicoke
100,M9R,"Kingsview Village, Richview Gardens, St. Phill...",Etobicoke
101,M9V,"Silverstone, South Steeles, Beaumond Heights, ...",Etobicoke


In [9]:
df1.shape

(103, 3)

In [10]:
link2="http://cocl.us/Geospatial_data"
tables2 = pd.read_csv(link2) # 0 represents the 1st table in the link and 1 represent the 2nd table

tables2
PC=pd.DataFrame(tables2)
PC

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [11]:
pc=pd.merge(df1,PC, on='Postal Code')
pc

,Postal Code,Neighbourhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Port Union, Highland Creek, Rouge Hill",Scarborough,43.784535,-79.160497
2,M1E,"West Hill, Morningside, Guildwood",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,Weston,York,43.706876,-79.518188
99,M9P,Westmount,Etobicoke,43.696319,-79.532242
100,M9R,"Kingsview Village, Richview Gardens, St. Phill...",Etobicoke,43.688905,-79.554724
101,M9V,"Silverstone, South Steeles, Beaumond Heights, ...",Etobicoke,43.739416,-79.588437


In [12]:
import numpy as np
import pandas as pd

import json
# !conda install -c conda-forge requests sklearn geopy --yes

from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans 
# !conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported')

Libraries imported


In [13]:
#pd.set_option('display max_columns',None)

#pd.set_option('display max_rows',None)

In [14]:
address='Toronto'
geolocator=Nominatim(user_agent="To_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical co-ordinates of Toronto are {},{}.'.format(latitude,longitude))


The geographical co-ordinates of Toronto are 43.653963,-79.387207.


In [15]:
map_Toronto=folium.Map(location=[43.653963,-79.387207],zoom_start=10)
for lat,lng,borough,neighborhood in zip(pc['Latitude'],pc['Longitude'],pc['Borough'],pc['Neighbourhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill='True',
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Toronto)
    
map_Toronto

In [16]:
CLIENT_ID='ZV2JW52JKPTQDUSXTD40M0YZY31SMGIUUMGUUYUUFTRXUINU'
CLIENT_SECRET='PCSWW4RIXAQAK50MIEP5ELQO04MN4OI0ZH3WFBE24AB3JZIR'
VERSION='20180604'
LIMIT= 30
print('Your credentials:')
print('Client_ID:' + CLIENT_ID)
print('Client_Secret:' + CLIENT_SECRET)

Your credentials:
Client_ID:ZV2JW52JKPTQDUSXTD40M0YZY31SMGIUUMGUUYUUFTRXUINU
Client_Secret:PCSWW4RIXAQAK50MIEP5ELQO04MN4OI0ZH3WFBE24AB3JZIR


In [17]:
PC_T=pc[pc['Borough'].str.contains("Toronto")]
PC_T=pd.DataFrame(PC_T)
PC_T

,Postal Code,Neighbourhood,Borough,Latitude,Longitude
37,M4E,The Beaches,East Toronto,43.676357,-79.293031
41,M4K,"The Danforth West, Riverdale",East Toronto,43.679557,-79.352188
42,M4L,"India Bazaar, The Beaches West",East Toronto,43.668999,-79.315572
43,M4M,Studio District,East Toronto,43.659526,-79.340923
44,M4N,Lawrence Park,Central Toronto,43.728020,-79.388790
45,M4P,Davisville North,Central Toronto,43.712751,-79.390197
46,M4R,North Toronto West,Central Toronto,43.715383,-79.405678
47,M4S,Davisville,Central Toronto,43.704324,-79.388790
48,M4T,"Moore Park, Summerhill East",Central Toronto,43.689574,-79.383160
49,M4V,"South Hill, Rathnelly, Forest Hill SE, Deer Pa...",Central Toronto,43.686412,-79.400049


In [18]:
# #PC_T.loc[0,'Neighbourhood']
# PC_1=PC_T['Neighbourhood'].iloc[0]
# PC_1

neighbourhood_lat=PC_T['Latitude'].iloc[0]
neighbourhood_long=PC_T['Longitude'].iloc[0]
neighbourhood_name=PC_T['Neighbourhood'].iloc[0]
print('Latitude and Longitude values of {} are {},{}.'.format(neighbourhood_name,
                                                             neighbourhood_lat,
                                                             neighbourhood_long))


In [19]:
# PC_lat=PC_T['Latitude']
# PC_long=PC_T['Longitude']
# PC_name=PC_T['Neighbourhood']
# print('Latitude and Longitude values of {} are {},{}.'.format(PC_name,
#                                                              PC_lat,
#                                                              PC_long))

In [20]:
# #LIMIT=100
# #radius=500
# #query= 'restaurants'
# #url=url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     PC_lat, 
#     PC_long, 
#     radius, 
#     LIMIT)


# #url

In [21]:
#results= requests.get(url).json()
#results

In [22]:
#venues=results['response']['venues']
#venues = results['response']['groups'][0]['items']
#nearby_venues=json_normalize(venues)

#dataframe.head()

#def get_category_type(row):
    try:
        categories_list= row['categories']
    except:
            categories_list= row['venue.categories']
            
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

venues= results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)

nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [23]:
#print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

venues_map = folium.Map(location=[neighbourhood_lat, neighbourhood_long], zoom_start=15)
# add a red circle marker to represent the BEDOKVILLE
folium.features.CircleMarker(
    [neighbourhood_lat, neighbourhood_long],
    radius=10,
    color='red',
    popup='BEDOKVILLE',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
# add all venues as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
venues_map

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)


In [25]:
Toronto_venues=getNearbyVenues(names= PC_T['Neighbourhood'],
                              latitudes= PC_T['Latitude'],
                              longitudes= PC_T['Longitude'])

Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [26]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",27,27,27,27,27,27
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Railway Lands, Harbourfront West, South Niagara, King and Spadina",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
Davisville,30,30,30,30,30,30
Davisville North,10,10,10,10,10,10


In [27]:
# Analyze each neighborhood
Toronto_onehot=pd.get_dummies(Toronto_venues[['Venue Category']],prefix="",prefix_sep="")
Toronto_onehot['Neighbourhood']=Toronto_venues['Neighborhood']
fixed_columns=[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot=Toronto_onehot[fixed_columns]
Toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
Toronto_grouped=Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074074
2,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632
3,"CN Tower, Bathurst Quay, Island airport, Railw...",0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
num_top_venues=5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp=Toronto_grouped[Toronto_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue                           freq
0                    Yoga Studio                    Yoga Studio
1                       Wine Bar                       Wine Bar
2          Vietnamese Restaurant          Vietnamese Restaurant
3               Video Game Store               Video Game Store
4  Vegetarian / Vegan Restaurant  Vegetarian / Vegan Restaurant


----Brockton, Parkdale Village, Exhibition Place----
                           venue                           freq
0                    Yoga Studio                    Yoga Studio
1                       Wine Bar                       Wine Bar
2          Vietnamese Restaurant          Vietnamese Restaurant
3               Video Game Store               Video Game Store
4  Vegetarian / Vegan Restaurant  Vegetarian / Vegan Restaurant


----Business Reply Mail Processing Centre 969 Eastern----
                           venue                           freq
0                    Yoga Studio 

4  Vegetarian / Vegan Restaurant  Vegetarian / Vegan Restaurant


----Underground city, First Canadian Place----
                           venue                           freq
0                    Yoga Studio                    Yoga Studio
1                       Wine Bar                       Wine Bar
2          Vietnamese Restaurant          Vietnamese Restaurant
3               Video Game Store               Video Game Store
4  Vegetarian / Vegan Restaurant  Vegetarian / Vegan Restaurant


----Victoria Hotel, Commerce Court----
                           venue                           freq
0                    Yoga Studio                    Yoga Studio
1                       Wine Bar                       Wine Bar
2          Vietnamese Restaurant          Vietnamese Restaurant
3               Video Game Store               Video Game Store
4  Vegetarian / Vegan Restaurant  Vegetarian / Vegan Restaurant




In [30]:
# Pandas dataframe
def return_most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [31]:
num_top_venues=10

indicators=['st','nd','rd']

columns=['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venues'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighbourhoods_venues_sorted=pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood']=Toronto_grouped['Neighbourhood']
for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:]=return_most_common_venues(Toronto_grouped.iloc[ind,:],num_top_venues)
    
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Beer Bar,Cocktail Bar,Seafood Restaurant,Breakfast Spot,Basketball Stadium,Italian Restaurant,Bistro,Fish Market,Clothing Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Yoga Studio,Bakery,Coffee Shop,Breakfast Spot,Furniture / Home Store,Intersection,Italian Restaurant,Convenience Store,Nightclub
2,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Restaurant,Burrito Place,Brewery,Fast Food Restaurant,Spa,Auto Workshop,Farmers Market,Pizza Place
3,"CN Tower, Bathurst Quay, Island airport, Railw...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Spa,Café,Bubble Tea Shop,Ramen Restaurant,Japanese Restaurant,Seafood Restaurant,Sandwich Place,Portuguese Restaurant


In [32]:
#Cluster neighborhoods 
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5
Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(Toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [33]:
neighbourhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

Toronto_merged = PC_T
Toronto_merged = Toronto_merged.merge(neighbourhoods_venues_sorted, on = 'Neighbourhood')

Toronto_merged.head()

,Postal Code,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,The Beaches,East Toronto,43.676357,-79.293031,0,Other Great Outdoors,Health Food Store,Trail,Pub,Neighborhood,Yoga Studio,Dance Studio,Event Space,Ethiopian Restaurant,Eastern European Restaurant
1,M4K,"The Danforth West, Riverdale",East Toronto,43.679557,-79.352188,0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Pub,Pizza Place,Coffee Shop
2,M4L,"India Bazaar, The Beaches West",East Toronto,43.668999,-79.315572,0,Park,Brewery,Sandwich Place,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse,Sushi Restaurant
3,M4M,Studio District,East Toronto,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,Latin American Restaurant,Bar,Fish Market,Neighborhood,Seafood Restaurant,Sandwich Place
4,M4N,Lawrence Park,Central Toronto,43.728020,-79.388790,1,Park,Swim School,Bus Line,Fish & Chips Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dog Run


In [34]:
# create map
map_clusters = folium.Map(location = [43.653963,-79.387207], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters
